# AWS Hail on EMR Bokeh Plotting Example

This is taken from the [Hail Plotting Tutorial](https://hail.is/docs/0.2/tutorials/08-plotting.html) with adjustments for use with SageMaker Notebook instances and EMR.

### List EMR Master Nodes

`~/SageMaker/bin/list-clusters` will output the IP of each master node in your account and check Livy connectivity.

In [ ]:
%%bash
~/SageMaker/bin/list-clusters

Use the Livy Endpoint above and start your session name `-s`, language `-l python`, the livy endpoint `-u`, and authentication type `-t`.

In [ ]:
%load_ext sparkmagic.magics
%spark add -s aperry -l python -u http://X.X.X.X:8998 -t None

### Plot Retrieval from Master

Set a local variable for the EMR Master IP.  http requests will be sent to the Master to pull the remote Hail plots back to this notebook server.  Notebook plots will be stored in a `plots` directory along side to your notebook.  If the plots directory does not exist, it will be created.

Once the plots have been pulled locally they will be shown inline.  

In [ ]:
%%local
emrMasterIp = 'X.X.X.X'

In [ ]:
%%local
from IPython.display import display, HTML
import requests, os

def displayRemoteHtml(remoteFile):
    url = 'http://' + emrMasterIp + remoteFile
    req = requests.get(url)
    
    # Strip leading /
    localPath = remoteFile[1:]
    # Drop the filename and create local target directory
    targetPath = "/".join(localPath.split('/')[0:-1])
    if not os.path.exists(targetPath):
        os.makedirs(targetPath, exist_ok=True)
    open(localPath, 'wb').write(req.content)
    
    html = HTML(filename=localPath)
    display(html)


## Plotting Tutorial

The Hail plot module allows for easy plotting of data. This notebook contains examples of how to use the plotting functions in this module, many of which can also be found in the first tutorial.

In [ ]:
# If using Hail version < 0.2.24, uncomment the following
# sc.addPyFile('/opt/hail/hail-python.zip')

import hail as hl
hl.init(sc)

from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

In [ ]:
hl.utils.get_1kg('data/')
mt = hl.read_matrix_table('data/1kg.mt')
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))
mt = mt.annotate_cols(**table[mt.s])
mt = hl.sample_qc(mt)

mt.describe()

### Histogram

The `histogram()` method takes as an argument an aggregated hist expression, as well as optional arguments for the legend and title of the plot.

In [ ]:
from bokeh.plotting import figure, output_file, save

dp_hist = mt.aggregate_entries(hl.expr.aggregators.hist(mt.DP, 0, 30, 30))
p = hl.plot.histogram(dp_hist, legend='DP', title='DP Histogram')
output_file("/plots/histogram3.html")
save(p)

In [ ]:
%%local
displayRemoteHtml('/plots/histogram3.html')

This method, like all Hail plotting methods, also allows us to pass in fields of our data set directly. Choosing not to specify the `range` and `bins` arguments would result in a range being computed based on the largest and smallest values in the dataset and a default bins value of 50.

### Cumulative Histogram

The `cumulative_histogram()` method works in a similar way to `histogram()`.

In [ ]:
p = hl.plot.cumulative_histogram(mt.DP, range=(0,30), bins=30)
output_file("/plots/cumulative_histogram.html")
save(p)

In [ ]:
%%local
displayRemoteHtml('/plots/cumulative_histogram.html')

### Scatter

The `scatter()` method can also take in either Python types or Hail fields as arguments for x and y.

In [ ]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
output_file("/plots/scatter.html")
save(p)

In [ ]:
%%local
displayRemoteHtml('/plots/scatter.html')

We can also pass in a Hail field as a `label` argument, which determines how to color the data points.

In [ ]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
ab = mt.AD[1] / hl.sum(mt.AD)
filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))
mt = mt.filter_entries(filter_condition_ab)
mt = hl.variant_qc(mt).cache()
common_mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)
gwas = hl.linear_regression_rows(y=common_mt.CaffeineConsumption, x=common_mt.GT.n_alt_alleles(), covariates=[1.0])
pca_eigenvalues, pca_scores, _ = hl.hwe_normalized_pca(common_mt.GT)

In [ ]:
p = hl.plot.scatter(pca_scores.scores[0], pca_scores.scores[1],
                    label=common_mt.cols()[pca_scores.s].SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2', collect_all=True)
output_file("/plots/scatter2.html")
save(p)

In [ ]:
%%local
displayRemoteHtml('/plots/scatter2.html')

Hail's downsample aggregator is incorporated into the `scatter()`, `qq()`, and `manhattan()` functions. The `collect_all` parameter tells the plot function whether to collect all values or downsample. Choosing not to set this parameter results in downsampling.

In [ ]:
p = hl.plot.scatter(pca_scores.scores[0], pca_scores.scores[1],
                    label=common_mt.cols()[pca_scores.s].SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2', collect_all=True)
p2 = hl.plot.scatter(pca_scores.scores[0], pca_scores.scores[1],
                    label=common_mt.cols()[pca_scores.s].SuperPopulation,
                    title='PCA (downsampled)', xlabel='PC1', ylabel='PC2', collect_all=False, n_divisions=50)

# show(gridplot([p, p2], ncols=2, plot_width=400, plot_height=400))
grid = gridplot([p, p2], ncols=2, plot_width=400, plot_height=400)
output_file("/plots/gridplot.html")
save(grid)


In [ ]:
%%local
displayRemoteHtml('/plots/gridplot.html')

### 2-D histogram

For visualizing relationships between variables in large datasets (where scatter plots may be less informative since they highlight outliers), the `histogram_2d()` function will create a heatmap with the number of observations in each section of a 2-d grid based on two variables.

In [ ]:
p = hl.plot.histogram2d(pca_scores.scores[0], pca_scores.scores[1])
output_file("/plots/2d-histogram.html")
save(p)

In [ ]:
%%local
displayRemoteHtml('/plots/2d-histogram.html')

### Q-Q (Quantile-Quantile)

The `qq()` function requires either a Python type or a Hail field containing p-values to be plotted. This function also allows for downsampling.

In [ ]:
p = hl.plot.qq(gwas.p_value, collect_all=True)
p2 = hl.plot.qq(gwas.p_value, n_divisions=75)

# show(gridplot([p, p2], ncols=2, plot_width=400, plot_height=400))
qq = gridplot([p, p2], ncols=2, plot_width=400, plot_height=400)
output_file("/plots/qq.html")
save(qq)

In [ ]:
%%local
displayRemoteHtml('/plots/qq.html')

### Manhattan

The `manhattan()` function requires a Hail field containing p-values.

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
output_file("/plots/manhattan.html")
save(p)

In [ ]:
%%local
displayRemoteHtml('/plots/manhattan.html')

We can also pass in a dictionary of fields that we would like to show up as we hover over a data point, and choose not to downsample if the dataset is relatively small.

In [ ]:
hover_fields = dict([('alleles', gwas.alleles)])
p = hl.plot.manhattan(gwas.p_value, hover_fields=hover_fields, collect_all=True)
output_file("/plots/manhattan_hover.html")
save(p)

In [ ]:
%%local
displayRemoteHtml('/plots/manhattan_hover.html')

Remove the Livy notebook session

In [ ]:
%spark cleanup